1. Supervised fine-tuning (SFT)
2. Annotating data with preference labels
3. Training a reward model on the preference data
4. RL optmization step

# Setting up params

In [1]:
# Logging to huggingface-cli

In [2]:
import typing_extensions

In [3]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

In [4]:
from datasets import load_dataset

data = load_dataset('squad', split='train')
data = data.to_pandas()[['id', 'context', 'title']]
data.drop_duplicates(subset='context', keep='first', inplace=True)
data.head()
     

Found cached dataset squad (/Users/chaum/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


,id,context,title
0,5733be284776f41900661182,"Architecturally, the school has a Catholic cha...",University_of_Notre_Dame
5,5733bf84d058e614000b61be,"As at most other universities, Notre Dame's st...",University_of_Notre_Dame
10,5733bed24776f41900661188,The university is the major seat of the Congre...,University_of_Notre_Dame
15,5733a6424776f41900660f51,The College of Engineering was established in ...,University_of_Notre_Dame
20,5733a70c4776f41900660f64,All of Notre Dame's undergraduate students are...,University_of_Notre_Dame


In [5]:
# from llama_index import GPTVectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import Document

In [6]:
# pip install datasets

In [7]:
from datasets import load_dataset

data = load_dataset('squad', split='train')
data = data.to_pandas()[['id', 'context', 'title']]
data.drop_duplicates(subset='context', keep='first', inplace=True)
data.head()

Found cached dataset squad (/Users/chaum/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


,id,context,title
0,5733be284776f41900661182,"Architecturally, the school has a Catholic cha...",University_of_Notre_Dame
5,5733bf84d058e614000b61be,"As at most other universities, Notre Dame's st...",University_of_Notre_Dame
10,5733bed24776f41900661188,The university is the major seat of the Congre...,University_of_Notre_Dame
15,5733a6424776f41900660f51,The College of Engineering was established in ...,University_of_Notre_Dame
20,5733a70c4776f41900660f64,All of Notre Dame's undergraduate students are...,University_of_Notre_Dame


In [8]:
model

'meta-llama/Llama-2-7b-chat-hf'

In [10]:
embeddings = HuggingFaceEmbedding(model_name=model)

: 

: 

In [ ]:


docs = []

for i, row in data.iterrows():
    docs.append(Document(
        text=row['context'],
        doc_id=row['id'],
        extra_info={'title': row['title']}
    ))
docs[0]
     